In [11]:
import requests
from datetime import datetime, timezone

sports = requests.get("https://gamma-api.polymarket.com/sports", timeout=20).json()
names = [item["sport"] for item in sports if isinstance(item, dict) and "sport" in item]
print(names)



['ncaab', 'epl', 'lal', 'cbb', 'ipl', 'wnba', 'bun', 'mlb', 'cfb', 'nfl', 'fl1', 'sea', 'ucl', 'afc', 'ofc', 'fif', 'ere', 'arg', 'itc', 'mex', 'lcs', 'lib', 'sud', 'tur', 'con', 'cof', 'uef', 'caf', 'rus', 'efa', 'efl', 'mls', 'nba', 'nhl', 'uel', 'csgo', 'dota2', 'lol', 'valorant', 'odi', 't20', 'abb', 'csa', 'atp', 'wta', 'cwbb', 'mma', 'cdr']


In [3]:
import requests, itertools

BASE = "https://gamma-api.polymarket.com"

def list_series(limit=100, offset=0, closed=None, order=None, ascending=None):
    params = {"limit": limit, "offset": offset}
    if closed is not None:
        params["closed"] = str(closed).lower()
    if order:
        params["order"] = order
    if ascending is not None:
        params["ascending"] = str(ascending).lower()
    r = requests.get(f"{BASE}/series", params=params, timeout=30)
    r.raise_for_status()
    return r.json()  # top-level LIST

series = list_series(limit=50)
print(f"Series returned: {len(series)}")

# Flatten the nested markets (events → markets), if present
def iter_markets_from_series(series_obj):
    for ev in series_obj.get("events", []) or []:
        for m in ev.get("markets", []) or []:
            yield m

all_markets = list(itertools.chain.from_iterable(
    iter_markets_from_series(s) for s in series if isinstance(s, dict)
))
print("Embedded markets found:", len(all_markets))
for m in all_markets[:5]:
    print("•", m.get("question"))


Series returned: 50
Embedded markets found: 0


In [21]:
def get_markets_by_tag_id(tag_id, closed=False, limit=250, max_pages=20):
    all_markets = []
    for page in range(max_pages):
        offset = page * limit
        params = {"tag_id": tag_id, "closed": str(closed).lower(), "limit": limit, "offset": offset}
        r = requests.get(f"{BASE}/markets", params=params, timeout=30)
        r.raise_for_status()
        chunk = r.json()
        markets = chunk if isinstance(chunk, list) else chunk.get("markets", chunk)
        # docs show /markets returns a list; some SDKs wrap it. Handle both.
        if not markets:
            break
        all_markets.extend(markets)
        if len(markets) < limit:
            break
    return all_markets

def get_alive_market(all_mkt):
    alive = [m for m in all_mkt if m.get('active') is True and m.get("acceptingOrders") is True]
    want_types = {"moneyline", "spread", "total"}  # example
    typed = [m for m in alive if str(m.get("sportsMarketType","")).lower() in want_types]
    print("Alive (selected types):", len(typed))
    return typed


In [23]:
sport_type = 'nhl'
spprt_tag = requests.get(f"{BASE}/tags/slug/{sport_type}", timeout=20).json()
sport_tag_id = int(spprt_tag["id"])    
all_mkt = get_markets_by_tag_id(sport_tag_id, closed=False, limit=250)
alive_mkt = get_alive_market(all_mkt)

Alive (selected types): 210


In [1]:
for i in range(10, -1, -1):
    print(i)

10
9
8
7
6
5
4
3
2
1
0


In [32]:
alive_mkt[0]

{'id': '605042',
 'question': 'Oilers vs. Flames',
 'conditionId': '0x1ec664b73bf312e2a2ca693deb94e126dd74b5dfa44bc9cedf1e8129ebe8652a',
 'slug': 'nhl-edm-cgy-2025-09-21',
 'resolutionSource': 'https://www.nhl.com/scores',
 'endDate': '2025-09-22T00:00:00Z',
 'liquidity': '5.9566',
 'startDate': '2025-09-20T06:22:02.212099Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/nhl.png',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/nhl.png',
 'description': 'In the upcoming NHL game, scheduled for September 21 at 8:00PM ET:\nIf the Oilers, the market will resolve to "Oilers".\nIf the Flames, the market will resolve to "Flames".\nIf the game is postponed, this market will remain open until the game has been completed.\nIf the game is canceled entirely, with no make-up game, this market will resolve 50-50.',
 'outcomes': '["Oilers", "Flames"]',
 'outcomePrices': '["0.5", "0.5"]',
 'volume': '510.615132',
 'active': True,
 'closed': False,
 'marketMakerAddress

In [35]:
import pandas as pd 

C:\Users\zhang\AppData\Local\Temp\ipykernel_8160\217833862.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [39]:
s1 = pd.Series({'a':1,'b':2} )
s2 = pd.Series({'a':3,'b':4} )

In [41]:
s1 

a    1
b    2
dtype: int64

In [43]:
pd.DataFrame([s1, s2])

,a,b
0,1,2
1,3,4


In [25]:

def add_formula_params(curr_df, midpoint, v, daily_reward):
    curr_df['s'] = (curr_df['price'] - midpoint).abs()
    curr_df['S'] = ((v - curr_df['s']) / v) ** 2
    curr_df['100'] = 1/curr_df['price'] * 100

    curr_df['size'] = curr_df['size'] + curr_df['100']

    curr_df['Q'] = curr_df['S'] * curr_df['size']
    curr_df['reward_per_100'] = (curr_df['Q'] / curr_df['Q'].sum()) * daily_reward / 2 / curr_df['size'] * curr_df['100']
    return curr_df

def process_single_row(row, client):
    ret = {}
    ret['question'] = row['question']
    ret['neg_risk'] = row['neg_risk']

    ret['answer1'] = row['tokens'][0]['outcome']
    ret['answer2'] = row['tokens'][1]['outcome']

    ret['min_size'] = row['rewards']['min_size']
    ret['max_spread'] = row['rewards']['max_spread']

    token1 = row['tokens'][0]['token_id']
    token2 = row['tokens'][1]['token_id']

    rate = 0
    for rate_info in row['rewards']['rates']:
        if rate_info['asset_address'].lower() == '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'.lower():
            rate = rate_info['rewards_daily_rate']
            break

    ret['rewards_daily_rate'] = rate
    book = client.get_order_book(token1)
    
    bids = pd.DataFrame()
    asks = pd.DataFrame()

    try:
        bids = pd.DataFrame(book.bids).astype(float)
    except:
        pass

    try:
        asks = pd.DataFrame(book.asks).astype(float)
    except:
        pass


    try:
        ret['best_bid'] = bids.iloc[-1]['price']
    except:
        ret['best_bid'] = 0

    try:
        ret['best_ask'] = asks.iloc[-1]['price']
    except:
        ret['best_ask'] = 0

    ret['midpoint'] = (ret['best_bid'] + ret['best_ask']) / 2
    
    TICK_SIZE = row['minimum_tick_size']
    ret['tick_size'] = TICK_SIZE

    bid_from, bid_to, ask_from, ask_to = get_bid_ask_range(ret, TICK_SIZE)
    v = round((ret['max_spread'] / 100), 2)

    bids_df = pd.DataFrame()
    bids_df['price'] = generate_numbers(bid_from, bid_to, TICK_SIZE)

    asks_df = pd.DataFrame()
    asks_df['price'] = generate_numbers(ask_from, ask_to, TICK_SIZE)

    try:
        bids_df = bids_df.merge(bids, on='price', how='left').fillna(0)
    except:
        bids_df = pd.DataFrame()

    try:
        asks_df = asks_df.merge(asks, on='price', how='left').fillna(0)
    except:
        asks_df = pd.DataFrame()

    best_bid_reward = 0
    ret_bid = pd.DataFrame()

    try:
        ret_bid = add_formula_params(bids_df, ret['midpoint'], v, rate)
        best_bid_reward = round(ret_bid['reward_per_100'].max(), 2)
    except:
        pass

    best_ask_reward = 0
    ret_ask = pd.DataFrame()

    try:
        ret_ask = add_formula_params(asks_df, ret['midpoint'], v, rate)
        best_ask_reward = round(ret_ask['reward_per_100'].max(), 2)
    except:
        pass

    ret['bid_reward_per_100'] = best_bid_reward
    ret['ask_reward_per_100'] = best_ask_reward

    ret['sm_reward_per_100'] = round((best_bid_reward + best_ask_reward) / 2, 2)
    ret['gm_reward_per_100'] = round((best_bid_reward * best_ask_reward) ** 0.5, 2)

    ret['end_date_iso'] = row['end_date_iso']
    ret['market_slug'] = row['market_slug']
    ret['token1'] = token1
    ret['token2'] = token2
    ret['condition_id'] = row['condition_id']

    return ret